In [1]:
import numpy as np
import time, os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
from cellpose import utils

import random
from skimage import io
from skimage.morphology import disk

import shutil
import tqdm
import napari
import pandas as pd
import mFISH3D.segment

import importlib

>>> GPU activated? 1


In [2]:
def get_file_prefix(directory,postfix):
    files = os.listdir(directory)
    files.sort()
    files = pd.Series(files)
    selected_files = files[files.str.contains(postfix)]
    prefixes = selected_files.str.split(postfix,expand=True)[0].to_list()
    
    return prefixes

In [3]:
def augmenter(x):
    """
    Augmentation of a single input/label image pair. 90rot, 180rot, 270rot, flip, flip90rot, flip180rot, flip270rot will be generated
    x is an input image
    """
    # Note that we only use fliprots along axis=(1,2), i.e. the yx axis 
    # as 3D microscopy acquisitions are usually not axially symmetric
    x90rot = np.rot90(x)
    x180rot = np.rot90(x90rot)
    x270rot = np.rot90(x180rot)
    xflip = np.flip(x)
    xflip90rot = np.rot90(xflip)
    xflip180rot = np.rot90(xflip90rot)
    xflip270rot = np.rot90(xflip180rot)
    
    return (x90rot, x180rot, x270rot, xflip, xflip90rot, xflip180rot, xflip270rot)

In [4]:
# make RGB image
dataset_folder = "/mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double"

postfix_list = ['img', 'imgnuc']

# read image and convert to RGB and save
prefixes = get_file_prefix(dataset_folder, postfix_list[0])
for prefix in prefixes:
    rgb = []
    for postfix in postfix_list:
        rgb.append(io.imread(os.path.join(dataset_folder,prefix+postfix+'.tif')))
    rgb = np.asarray(rgb)
    rgb = np.moveaxis(rgb, 0, -1)
    rgb = np.pad(rgb, ((0, 0), (0, 0), (0, 3-len(postfix_list))), 'constant')
    io.imsave(os.path.join(dataset_folder,prefix+'rgb'+'.tif'),rgb)

/tmp/ipykernel_2705460/678726030.py:15: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0000_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+'rgb'+'.tif'),rgb)
/tmp/ipykernel_2705460/678726030.py:15: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0001_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+'rgb'+'.tif'),rgb)
/tmp/ipykernel_2705460/678726030.py:15: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0002_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+'rgb'+'.tif'),rgb)
/tmp/ipykernel_2705460/678726030.py:15: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0003_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+'rgb'+'.tif'),r

In [5]:
# path to dataset

# other parameters for training.
number_of_epochs =  500

Training_channel = 1
Second_training_channel = 2

batch_size =  8
initial_learning_rate = 0.0002

img_postfix = 'rgb'
mask_postfix = 'mask'
extension = '.tif'

In [6]:
# data argumentation
prefixes = get_file_prefix(dataset_folder, img_postfix)
for prefix in prefixes:
    x = io.imread(os.path.join(dataset_folder,prefix+img_postfix+extension))
    y = io.imread(os.path.join(dataset_folder,prefix+mask_postfix+extension))
    x_args = augmenter(x)
    y_args = augmenter(y)
    counter = 0
    for x_arg, y_arg in zip(x_args, y_args):
        io.imsave(os.path.join(dataset_folder,prefix+str(counter)+'arg_'+img_postfix+extension),x_arg)
        io.imsave(os.path.join(dataset_folder,prefix+str(counter)+'arg_'+mask_postfix+extension),y_arg)
        counter += 1

/tmp/ipykernel_2705460/2463502447.py:10: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0000_0arg_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+str(counter)+'arg_'+img_postfix+extension),x_arg)
/tmp/ipykernel_2705460/2463502447.py:11: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0000_0arg_mask.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+str(counter)+'arg_'+mask_postfix+extension),y_arg)
/tmp/ipykernel_2705460/2463502447.py:10: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/slc17a7_double1_0000_1arg_rgb.tif is a low contrast image
  io.imsave(os.path.join(dataset_folder,prefix+str(counter)+'arg_'+img_postfix+extension),x_arg)
/tmp/ipykernel_2705460/2463502447.py:11: UserWarning: /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/s

In [7]:
# data normalization if necessary
# prefixes = get_file_prefix(dataset_folder, img_postfix)
# for prefix in prefixes:
#     x = io.imread(os.path.join(dataset_folder,prefix+img_postfix+extension))
#     x_norm = mFISH3D.segment.gpu_percentile_normalization(x, footprint=np.ones((50,50)))
#     io.imsave(os.path.join(dataset_folder,prefix+img_postfix+extension),x_norm)

In [7]:
train_folder = os.path.join(dataset_folder,'training')
test_folder = os.path.join(dataset_folder,'test')

if not os.path.isdir(train_folder):
    os.makedirs(train_folder)
if not os.path.isdir(test_folder):
    os.makedirs(test_folder)

In [8]:
prefixes = get_file_prefix(dataset_folder, img_postfix)

rng = np.random.RandomState(42)
ind = rng.permutation(len(prefixes))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
for i in ind_val:
    os.rename(os.path.join(dataset_folder,prefixes[i]+img_postfix+extension),os.path.join(test_folder,prefixes[i]+img_postfix+extension))
    os.rename(os.path.join(dataset_folder,prefixes[i]+mask_postfix+extension),os.path.join(test_folder,prefixes[i]+mask_postfix+extension))
for i in ind_train:
    os.rename(os.path.join(dataset_folder,prefixes[i]+img_postfix+extension),os.path.join(train_folder,prefixes[i]+img_postfix+extension))
    os.rename(os.path.join(dataset_folder,prefixes[i]+mask_postfix+extension),os.path.join(train_folder,prefixes[i]+mask_postfix+extension))


In [9]:
### set pretrained model if there is any
model_to_load = None#'/home/tmurakami/src/misc_notebooks/temp/cellpose_models/train_folder/models/cellpose_residual_on_style_on_concatenation_off_train_folder_2022_07_12_09_41_24.342858'

In [10]:
%%time

# https://github.com/MouseLand/cellpose/issues/54
# https://github.com/MouseLand/cellpose/issues/412
!python -m cellpose --train --use_gpu --dir $train_folder --test_dir $test_folder --pretrained_model $model_to_load --chan $Training_channel --chan2 $Second_training_channel --n_epochs $number_of_epochs --learning_rate $initial_learning_rate --batch_size $batch_size --img_filter $img_postfix --mask_filter $mask_postfix  --verbose


2022-08-27 16:18:16,439 [INFO] WRITING LOG OUTPUT TO /home/tmurakami/.cellpose/run.log
2022-08-27 16:18:17,463 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 16:18:17,463 [INFO] >>>> using GPU
2022-08-27 16:18:17,495 [INFO] not all flows are present, running flow generation for all images
2022-08-27 16:18:17,967 [INFO] 204 / 204 images in /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/training folder have labels
2022-08-27 16:18:17,973 [INFO] not all flows are present, running flow generation for all images
2022-08-27 16:18:18,054 [INFO] 36 / 36 images in /mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/slc17a7_double/test folder have labels
2022-08-27 16:18:18,055 [INFO] >>>> during training rescaling images to fixed diameter of 30.0 pixels
/home/tmurakami/app/miniconda3/envs/cellpose/lib/python3.8/site-packages/cellpose/transforms.py:352: UserWarning: chan to seg' has value range of ZERO
  warnings.warn("chan to seg' h

In [12]:
# !python -m cellpose --train --use_gpu --anisotropy 4 --dir $train_folder --test_dir $test_folder --pretrained_model $model_to_load --chan 0 --chan2 0 --n_epochs $number_of_epochs --learning_rate $initial_learning_rate --batch_size $batch_size --img_filter img --mask_filter masks
# !python -m cellpose --train --use_gpu --fast_mode --dir $train_folder --test_dir $test_folder --pretrained_model $model_to_load --chan $Training_channel --chan2 $Second_training_channel --n_epochs $number_of_epochs --learning_rate $initial_learning_rate --batch_size $batch_size --img_filter img --mask_filter masks
# !python -m cellpose --train --use_gpu --dir $train_folder --test_dir $test_folder --pretrained_model $model_to_load --chan $Training_channel --chan2 $Second_training_channel --n_epochs $number_of_epochs --learning_rate $initial_learning_rate --batch_size $batch_size --img_filter img --mask_filter masks

In [5]:
# do some validation with validation dataset

In [10]:
models_path = os.path.join(train_folder,'models')
models_file = os.listdir(models_path); models_file.sort()
model_path = os.path.join(train_folder,'models',models_file[-1])

model = models.CellposeModel(gpu=use_GPU, pretrained_model=model_path, net_avg=False) #torch=True, diam_mean=10, do_3D=True, net_avg=True, device=None, residual_on=True, style_on=True, concatenation=False)

In [11]:
# load validation data
# img_3D = io.imread('/home/tmurakami/src/misc_notebooks/temp/cellpose_models/validation/slc17a7.tif')
img_3D = io.imread('/mnt/ampa_data01/tmurakami/conf_proc/human_ish_training_dataset/validation/plp1_01.tif')

In [12]:
%%time
# masks, flows, styles = model.eval(img_3D, channels=[0,0], diameter=10, do_3D=True, min_size=100, cellprob_threshold=3, progress=True) # nuclei??
# masks, flows, styles = model.eval(img_3D, channels=[0,0], z_axis=0, diameter=10, do_3D=True, min_size=100, cellprob_threshold=0.0, progress=True) # anisotropy=3.0/1.5
# for the faster segmentation, adjust diameter. The larger the faster.
pre_masks, _, _ = model.eval(img_3D, channels=[0,0], z_axis=0, do_3D=False, min_size=100, cellprob_threshold=0.0, progress=True, stitch_threshold=0.3) # 

print('The first segmentation is done.')
# get local max
interpolator = mFISH3D.segment.get_cellular_intensity_interpolator(pre_masks,img_3D)
local_max = mFISH3D.segment.local_max_with_interpolator(interpolator,img_3D.shape)

# normalize using local max before second segmentation
img_3D_norm = mFISH3D.segment.gpu_percentile_normalization(img_3D, footprint=np.ones((1,5,100)), img_high=local_max)

print('local normalization is done.')

masks, flows, styles = model.eval(img_3D_norm, channels=[0,0], z_axis=0, diameter=10, do_3D=True, min_size=100, cellprob_threshold=0.0, progress=True)
print('The second segmentation is done.')

The first segmentation is done.
local normalization is done.
The second segmentation is done.
CPU times: user 1min 41s, sys: 24.3 s, total: 2min 6s
Wall time: 1min


In [13]:
viewer = napari.Viewer()
viewer.add_image(img_3D, rgb=False, name='image01' , colormap='gray' ,blending='additive')
viewer.add_labels(pre_masks)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x7f2d9acc5c10>

In [14]:
#